In [4]:
#install necessary libraries
%pip install rdkit
%pip install mordredcommunity
%pip install propy3

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [16]:
#import necessary libraries
from pathlib import Path
import pandas as pd
import numpy as np 
from propy import PyPro
from rdkit import Chem
from mordred import Calculator, descriptors
from rdkit.Chem import rdFingerprintGenerator
from rdkit.Chem.rdmolops import GetAdjacencyMatrix



In [6]:
#USE THIS WHEN CONVERTING IN PYTHON SCRIPT
#HERE = Path(__file__).resolve().parent if "__file__" in globals() else Path.cwd()
#csv_path = HERE / "ligands_pubchem.csv"          # put the CSV next to the notebook
#lig_kinetics = pd.read_csv(csv_path)

In [17]:
#load necessary data
dir_DATA = "C:/Users/Pauli/OneDrive/Dokumente/Uni/Masterarbeit/code/preprocessing/"
lig_kinetics = pd.read_csv(f"{dir_DATA}/ligands_pubchem.csv", index_col=0, header=0)
df_kinases = pd.read_excel(dir_DATA+"klifs_sequence_pdbids.xlsx")



In [18]:
lig_kinetics

,ligand_name,CAS,ligand,commercial_name,SMILES
0,3-Methyladenine,5142-23-4,3-Methyladenine,3-Methyladenine,CN1C=NC(=N)C2=C1N=CN2
1,A66,1166227-08-2,A66,A66,CC1=C(SC(=N1)NC(=O)N2CCC[C@H]2C(=O)N)C3=CSC(=N...
2,A-674563,552325-73-2,A-674563,A-674563,CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OC[C@H](CC4=C...
3,A-769662,844499-71-4,A-769662,A-769662,C1=CC=C(C(=C1)C2=CC=C(C=C2)C3=CSC4=C3C(=C(C(=O...
4,AEE788 (NVP-AEE788),497839-62-0,AEE788,NVP-AEE788,CCN1CCN(CC1)CC2=CC=C(C=C2)C3=CC4=C(N3)N=CN=C4N...
...,...,...,...,...,...
265,Y-27632 2HCl,129830-38-2,Y-27632 2HCl,Y-27632 2HCl,C[C@H](C1CCC(CC1)C(=O)NC2=CC=NC=C2)N.Cl.Cl
266,YM201636,371942-69-7,YM201636,YM201636,C1COCCN1C2=NC(=NC3=C2OC4=C3C=CC=N4)C5=CC(=CC=C...
267,ZM 336372,208260-29-1,ZM 336372,ZM 336372,CC1=C(C=C(C=C1)NC(=O)C2=CC(=CC=C2)N(C)C)NC(=O)...
268,ZM-447439,331771-20-1,ZM-447439,ZM-447439,COC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=C(C=C3)NC(=O)...


Calculate Mol Obj from SMILES

In [3]:
#Third-party functions from Group Member Iris Guo
def cansmi_getmol(row, smi_col='smiles'):
    ori_smi = row[smi_col]
    try:
        mol = Chem.MolFromSmiles(ori_smi)
        can_smi = Chem.MolToSmiles(mol)
        return(can_smi, mol)
    except:
        return(None, None)

def remove_parsefail(df, molcol="mol_obj"):
    mask = df[molcol].isna()
    parsefail = df.loc[mask,:]
    df_updt = df.loc[~mask, :]
    return(df_updt, parsefail)

lig_kinetics[['smi', 'mol_obj']] = lig_kinetics.apply(cansmi_getmol, args=('SMILES', ), axis='columns', result_type='expand')
lig_kinetics, lig_kinetics_parsefail = remove_parsefail(lig_kinetics)

#save updated dataframe
#lig_kinetics.to_csv(f"{dir_DATA}/ligands_pubchem_withmol.csv", index=True)

Calculate Ligand Features

In [4]:
#rename the index to ligand_id
lig_kinetics.index.name = "ligand_id"
lig_kinetics 

,ligand_name,CAS,ligand,commercial_name,SMILES,smi,mol_obj
ligand_id,,,,,,,
0,3-Methyladenine,5142-23-4,3-Methyladenine,3-Methyladenine,CN1C=NC(=N)C2=C1N=CN2,Cn1cnc(=N)c2[nH]cnc21,<rdkit.Chem.rdchem.Mol object at 0x000002A9382...
1,A66,1166227-08-2,A66,A66,CC1=C(SC(=N1)NC(=O)N2CCC[C@H]2C(=O)N)C3=CSC(=N...,Cc1nc(NC(=O)N2CCC[C@H]2C(N)=O)sc1-c1csc(C(C)(C...,<rdkit.Chem.rdchem.Mol object at 0x000002A9382...
2,A-674563,552325-73-2,A-674563,A-674563,CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OC[C@H](CC4=C...,Cc1[nH]nc2ccc(-c3cncc(OC[C@@H](N)Cc4ccccc4)c3)...,<rdkit.Chem.rdchem.Mol object at 0x000002A9382...
3,A-769662,844499-71-4,A-769662,A-769662,C1=CC=C(C(=C1)C2=CC=C(C=C2)C3=CSC4=C3C(=C(C(=O...,N#Cc1c(O)c2c(-c3ccc(-c4ccccc4O)cc3)csc2[nH]c1=O,<rdkit.Chem.rdchem.Mol object at 0x000002A9382...
4,AEE788 (NVP-AEE788),497839-62-0,AEE788,NVP-AEE788,CCN1CCN(CC1)CC2=CC=C(C=C2)C3=CC4=C(N3)N=CN=C4N...,CCN1CCN(Cc2ccc(-c3cc4c(N[C@H](C)c5ccccc5)ncnc4...,<rdkit.Chem.rdchem.Mol object at 0x000002A9382...
...,...,...,...,...,...,...,...
265,Y-27632 2HCl,129830-38-2,Y-27632 2HCl,Y-27632 2HCl,C[C@H](C1CCC(CC1)C(=O)NC2=CC=NC=C2)N.Cl.Cl,C[C@@H](N)C1CCC(C(=O)Nc2ccncc2)CC1.Cl.Cl,<rdkit.Chem.rdchem.Mol object at 0x000002A9383...
266,YM201636,371942-69-7,YM201636,YM201636,C1COCCN1C2=NC(=NC3=C2OC4=C3C=CC=N4)C5=CC(=CC=C...,Nc1ccc(C(=O)Nc2cccc(-c3nc(N4CCOCC4)c4oc5ncccc5...,<rdkit.Chem.rdchem.Mol object at 0x000002A9383...
267,ZM 336372,208260-29-1,ZM 336372,ZM 336372,CC1=C(C=C(C=C1)NC(=O)C2=CC(=CC=C2)N(C)C)NC(=O)...,Cc1ccc(NC(=O)c2cccc(N(C)C)c2)cc1NC(=O)c1ccc(O)cc1,<rdkit.Chem.rdchem.Mol object at 0x000002A9383...


Calculate Mordred Features

In [5]:
# calculate Mordred features
mordred_calc = Calculator(descriptors, ignore_3D=True)
mordred_feat_df = mordred_calc.pandas(lig_kinetics.mol_obj.to_list())

100%|██████████| 270/270 [00:13<00:00, 20.77it/s]


In [6]:
#set index to ligand id
mordred_feat_df.index = lig_kinetics.index
# insert also the ligand name  from row "ligand" for easier identification
mordred_feat_df.insert(0, "ligand", lig_kinetics["ligand_name"])
mordred_feat_df 

,ligand,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
ligand_id,,,,,,,,,,,,,,,,,,,,,
0,3-Methyladenine,8.582741,8.124951,0,0,14.300805,2.41022,4.728234,14.300805,1.300073,...,9.362632,55.477134,149.070145,8.281675,136,15,58.0,69.0,3.694444,2.416667
1,A66,20.449495,17.975646,0,0,31.715466,2.452664,4.715426,31.715466,1.219826,...,10.176830,78.066147,393.129317,8.023047,1796,38,140.0,165.0,10.3125,5.472222
2,A-674563,21.310663,15.946662,0,1,35.97119,2.435929,4.804847,35.97119,1.332266,...,10.110136,75.964825,358.179361,7.309783,2136,39,142.0,165.0,7.138889,5.916667
3,A-769662,20.510745,16.154131,0,0,34.025121,2.514928,4.94777,34.025121,1.308658,...,10.340742,75.452185,360.056863,9.475181,1708,46,142.0,173.0,8.111111,5.722222
4,AEE788 (NVP-AEE788),26.110580,17.966093,0,2,44.278963,2.465847,4.801454,44.278963,1.341787,...,10.366750,83.177122,440.268845,6.773367,3773,51,176.0,207.0,8.361111,7.277778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,Y-27632 2HCl,13.682318,11.284539,0,1,multiple fragments (SpAbs_A/SpAbs),multiple fragments (SpMax_A/SpMax),multiple fragments (SpDiam_A/SpDiam),multiple fragments (SpAD_A/SpAD),multiple fragments (SpMAD_A/SpMAD),...,9.527994,52.474918,319.121818,7.421438,3700000677,24,88.0,99.0,divide by zero encountered in power (mZagreb1),4.055556
266,YM201636,28.219970,19.979760,0,0,47.215831,2.544505,4.989203,47.215831,1.349024,...,10.582308,86.196768,467.170588,8.342332,3815,59,194.0,233.0,8.583333,7.527778
267,ZM 336372,22.414889,17.320789,0,0,36.721613,2.363586,4.727173,36.721613,1.266263,...,10.152806,64.251724,389.173942,7.484114,2526,44,148.0,170.0,10.361111,6.361111


Calculate Morgan Fingerprints

In [7]:
# calculate fingerprints

mols = lig_kinetics.mol_obj.to_list()
ligands_ids = lig_kinetics.index #store ligand ids for later use

morgan_fpgen = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=2048)
fp_list = []
fps_bitinfo = []
ao = rdFingerprintGenerator.AdditionalOutput()
ao.AllocateBitInfoMap()
for m in mols:
    fp_list.append(morgan_fpgen.GetFingerprintAsNumPy(m, additionalOutput=ao))
    fps_bitinfo.append(ao.GetBitInfoMap())
morgan_fps = np.vstack(fp_list)

# create dataframe with fingerprints
morgan_fps_df = pd.DataFrame(morgan_fps, index=ligands_ids)
#add prefix to column names
morgan_fps_df=morgan_fps_df.add_prefix('morgan__')
# add column for ligand name
morgan_fps_df.insert(0, "ligand", lig_kinetics["ligand_name"])
morgan_fps_df 

,ligand,morgan__0,morgan__1,morgan__2,morgan__3,morgan__4,morgan__5,morgan__6,morgan__7,morgan__8,...,morgan__2038,morgan__2039,morgan__2040,morgan__2041,morgan__2042,morgan__2043,morgan__2044,morgan__2045,morgan__2046,morgan__2047
ligand_id,,,,,,,,,,,,,,,,,,,,,
0,3-Methyladenine,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,A66,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,A-674563,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,A-769662,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AEE788 (NVP-AEE788),0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,Y-27632 2HCl,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
266,YM201636,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
267,ZM 336372,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Graph Neural Network Adjacency matrix and features generation

In [8]:
# Raw adjacency (with self-loops)
def mol_to_adj_with_loops(m):
    if m is None:
        return None
    A = GetAdjacencyMatrix(m).astype(np.float32)
    # add self loops
    np.fill_diagonal(A, 1.0)
    return A


# Kipf normalization  A_hat = D^{-1/2} A D^{-1/2}
def kipf_norm(A):
    if A is None:
        return None
    # degree vector
    d = A.sum(axis=1)
    # guard against divide-by-zero
    d = np.where(d == 0, 1.0, d).astype(np.float32)
    D_inv_sqrt = np.diag(1.0 / np.sqrt(d))
    return (D_inv_sqrt @ A @ D_inv_sqrt).astype(np.float32)

# Create adjacency matrix dataframe 
mols = lig_kinetics.mol_obj.to_list()
df_adj = pd.DataFrame(mols, columns=["mol_obj"], index=lig_kinetics.index)
#insert also the ligand name  from row "ligand" for easier identification
df_adj.insert(0, "ligand", lig_kinetics["ligand_name"])
#apply functions to create adjacency matrices
df_adj["adjacency_raw"]=df_adj["mol_obj"].apply(mol_to_adj_with_loops)
df_adj["adjacency_kipf"]=df_adj["adjacency_raw"].apply(kipf_norm)
df_adj

,ligand,mol_obj,adjacency_raw,adjacency_kipf
ligand_id,,,,
0,3-Methyladenine,<rdkit.Chem.rdchem.Mol object at 0x000002A9382...,"[[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.49999997, 0.35355338, 0.0, 0.0, 0.0, 0.0, ..."
1,A66,<rdkit.Chem.rdchem.Mol object at 0x000002A9382...,"[[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.49999997, 0.35355338, 0.0, 0.0, 0.0, 0.0, ..."
2,A-674563,<rdkit.Chem.rdchem.Mol object at 0x000002A9382...,"[[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.49999997, 0.35355338, 0.0, 0.0, 0.0, 0.0, ..."
3,A-769662,<rdkit.Chem.rdchem.Mol object at 0x000002A9382...,"[[1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,...","[[0.3333333, 0.3333333, 0.0, 0.0, 0.0, 0.33333..."
4,AEE788 (NVP-AEE788),<rdkit.Chem.rdchem.Mol object at 0x000002A9382...,"[[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.49999997, 0.40824828, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...
265,Y-27632 2HCl,<rdkit.Chem.rdchem.Mol object at 0x000002A9383...,"[[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.49999997, 0.35355338, 0.0, 0.0, 0.0, 0.0, ..."
266,YM201636,<rdkit.Chem.rdchem.Mol object at 0x000002A9383...,"[[1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,...","[[0.3333333, 0.3333333, 0.0, 0.0, 0.0, 0.28867..."
267,ZM 336372,<rdkit.Chem.rdchem.Mol object at 0x000002A9383...,"[[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.49999997, 0.35355338, 0.0, 0.0, 0.0, 0.0, ..."


In [9]:
# code adapted from https://github.com/kochgroup/intro_pharma_ai/blob/main/utils/onehotencoder.py
from rdkit import Chem

def fetch_categories(mols, feat):
    """Function to retrieve all existing values for the desired atomic feature.

    Args in:
        mols (list): list of Molecules in rdkit mol format.
        feat (str): the atomic feature requested.

    Arg out:
        cat_list (list): list of all values of the atomic feature present in
            the data set.
    """
    cat_list = []
    for m in mols:
        for atom in m.GetAtoms():
            cat = eval(f"atom.Get{feat}()")
            if cat not in cat_list:
                cat_list.append(cat)
    cat_list.sort()
    return(cat_list)

def mol2onehot_atom(m, cat_list, feat):
    """Function to one-hot encode a molecule [m] with categories [cat_list]

    Args in:
        m (rdkit mol): Molecule to be transformed into a graph.
        cat_list (list): list of all values of the atomic feature present in
            the data set. The length of the list will determine the first
            dimension of the feature matrix.
        feat (str): the atomic feature requested.

    Args out:
        features (numpy array): one-hot-encoded feature matrix.
    """
    # initiate the one-hot vector
    features = np.empty((m.GetNumAtoms(), len(cat_list)), dtype=type(cat_list[0]))
    # assign binary values based on the identity of each atom
    # the .GetAtoms() function retrieves the list of atoms which
    # (unless specifically manipulated) has the same order as the SMILES string
    # corresponding to the mol object
    for i, a in enumerate(m.GetAtoms()):
        features[i] = eval(f"a.Get{feat}()")
    features = np.where(features == np.tile(cat_list, (m.GetNumAtoms(), 1)), 1, 0)
    return features

def onehotencode_atoms(mols):
    """Function to one-hot encode the atoms.

    Arg in:
        mols(list): list of Molecules in rdkit mol format.
        [global]calc_features(list) : list of feature names to be calculated by
            rdkit.

    Args out:
        OHEM(list): one-hot encoded features.
        cat_dict (dictionary): collection of all existing values for each atomic
            feature.
    """
    # retrieve all existing atom types in the given set of molecules
    cat_dict = {}
    for j in range(len(calc_features)):
        feat = calc_features[j]
        cat_list = fetch_categories(mols, feat)
        cat_dict[feat] = cat_list

        # create one-hot representation of each molecule
        if j == 0:
            OHEM = [mol2onehot_atom(m, cat_list, feat) for m in mols]
        else:
            OHEM = [np.concatenate((OHEM[k], mol2onehot_atom(mols[k], cat_list, feat)), axis=1) for k in range(len(mols))]


    return OHEM, cat_dict


# the commands to retrieve atomic properties from rdkit
# always take the form of atom.Get[Property]
# store the desired [Property] values in the list to be called
# options can be found at https://www.rdkit.org/docs/source/rdkit.Chem.rdchem.html
calc_features = ["AtomicNum", "FormalCharge", "IsAromatic",
                 "ExplicitValence", "TotalNumHs", "Hybridization"]  # "IsInRing" was remmoved because it caused problems

#FM, cat_dict = onehotencode_atoms(mols)

# check if the test set molecules contain any atom that does not appear in the training set
# (if so, those entries need to be removed)
#print(f"the training and test set molecules have the same list of feature categories:\n{cat_dict_train==cat_dict_test}\n")
# display all available categories
#print(cat_dict)

In [10]:
# Build per-atom feature matrices with your existing helper
# NOTE: onehotencode_atoms expects a list of mols and returns (list_of_feature_mats, cat_dict)
mols_list = df_adj["mol_obj"].tolist()
FM_list, cat_dict = onehotencode_atoms(mols_list)  # uses your existing code
df_adj["feature_matrix"] = FM_list
df_adj

[15:12:25] DEPRECATION WARNING: please use GetValence(which=)
[15:12:25] DEPRECATION WARNING: please use GetValence(which=)
[15:12:25] DEPRECATION WARNING: please use GetValence(which=)
[15:12:25] DEPRECATION WARNING: please use GetValence(which=)
[15:12:25] DEPRECATION WARNING: please use GetValence(which=)
[15:12:25] DEPRECATION WARNING: please use GetValence(which=)
[15:12:25] DEPRECATION WARNING: please use GetValence(which=)
[15:12:25] DEPRECATION WARNING: please use GetValence(which=)
[15:12:25] DEPRECATION WARNING: please use GetValence(which=)
[15:12:25] DEPRECATION WARNING: please use GetValence(which=)
[15:12:25] DEPRECATION WARNING: please use GetValence(which=)
[15:12:25] DEPRECATION WARNING: please use GetValence(which=)
[15:12:25] DEPRECATION WARNING: please use GetValence(which=)
[15:12:25] DEPRECATION WARNING: please use GetValence(which=)
[15:12:25] DEPRECATION WARNING: please use GetValence(which=)
[15:12:25] DEPRECATION WARNING: please use GetValence(which=)
[15:12:2

,ligand,mol_obj,adjacency_raw,adjacency_kipf,feature_matrix
ligand_id,,,,,
0,3-Methyladenine,<rdkit.Chem.rdchem.Mol object at 0x000002A9382...,"[[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.49999997, 0.35355338, 0.0, 0.0, 0.0, 0.0, ...","[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,..."
1,A66,<rdkit.Chem.rdchem.Mol object at 0x000002A9382...,"[[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.49999997, 0.35355338, 0.0, 0.0, 0.0, 0.0, ...","[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,..."
2,A-674563,<rdkit.Chem.rdchem.Mol object at 0x000002A9382...,"[[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.49999997, 0.35355338, 0.0, 0.0, 0.0, 0.0, ...","[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,..."
3,A-769662,<rdkit.Chem.rdchem.Mol object at 0x000002A9382...,"[[1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,...","[[0.3333333, 0.3333333, 0.0, 0.0, 0.0, 0.33333...","[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,..."
4,AEE788 (NVP-AEE788),<rdkit.Chem.rdchem.Mol object at 0x000002A9382...,"[[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.49999997, 0.40824828, 0.0, 0.0, 0.0, 0.0, ...","[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,..."
...,...,...,...,...,...
265,Y-27632 2HCl,<rdkit.Chem.rdchem.Mol object at 0x000002A9383...,"[[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.49999997, 0.35355338, 0.0, 0.0, 0.0, 0.0, ...","[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,..."
266,YM201636,<rdkit.Chem.rdchem.Mol object at 0x000002A9383...,"[[1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,...","[[0.3333333, 0.3333333, 0.0, 0.0, 0.0, 0.28867...","[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,..."
267,ZM 336372,<rdkit.Chem.rdchem.Mol object at 0x000002A9383...,"[[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.49999997, 0.35355338, 0.0, 0.0, 0.0, 0.0, ...","[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,..."


In [11]:
df_kinases

,kinase.klifs_id,kinase_name,kinase.klifs_name,kinase.full_name,kinase.gene_name,kinase.group,kinase.uniprot,structure.pdb_id,mutation,structure.klifs_id,structure.missing_residues,structure.pocket,Unnamed: 12,Unnamed: 13,n_res_kinetics_assay,UniProt_ID,Protein_Sequence
0,392,ABL,ABL1,"ABL proto-oncogene 1, non-receptor tyrosine ki...",ABL1,TYR,P00519,6xr7,NaN,12883,0,HKLGGGQYGEVYEVAVKTLEFLKEAAVMKEIKPNLVQLLGVYIITE...,NaN,NaN,1129,P00519,MLEICLKLVGCKSKKGLSSSSSCYLEEALQRPVASDFEPQGLSEAA...
1,1,AKT1,AKT1,v-akt murine thymoma viral oncogene homolog 1,AKT1,AGC,P31749,6buu,NaN,10432,0,KLLGKGTFGKVILYAMKILHTLTENRVLQNSRPFLTALKYSCFVME...,NaN,NaN,377,P31749,MSDVAIVKEGWLHKRGEYIKTWRPRYFLLKNDGTFIGYKERPQDVD...
2,396,ALK,ALK,anaplastic lymphoma receptor tyrosine kinase,ALK,TYR,Q9UM73,5ftq,NaN,6519,0,RGLGHGAFGEVYEVAVKTLDFLMEALIISKFNQNIVRCIGVFILLE...,NaN,NaN,563,Q9UM73,MGAIGLLWLLPLLLSTAAVGSGMGTGQRAGSPAAGPPLQPREPLSY...
3,259,AurA,AurA,aurora kinase A,AURKA,CAMK,O14965,6vpj,NaN,12657,0,RPLGKGKFGNVYLLALKVLQLRREVEIQSHLRPNILRLYGYYLILE...,NaN,NaN,403,O14965,MDRSKENCISGPVKATAPVGGPKRVLVTQQFPCQNPLPVNSGQAQR...
4,398,AXL,AXL,AXL receptor tyrosine kinase,AXL,TYR,P30530,5u6b,NaN,8260,0,KTLGEGEFGAVMEVAVKTMDFLSEAVCMKEFDPNVMRLIGVVVILP...,NaN,NaN,422,P30530,MAWRCPRMGRVPLAWCLALCGWACMAPRGTQAEESPFVGNPGNITG...
5,509,BRAF,BRAF,"B-Raf proto-oncogene, serine/threonine kinase",BRAF,TKL,P15056,6n0p,NaN,11597,0,QRIGSGSFGTVYKVAVKMLAFKNEVGVLRKTRVNILLFMGYAIVTQ...,NaN,NaN,294,P15056,MAALSGGGGGGAEPGQALFNGDMEPEAGAGAGAAASSAADPAIPEE...
6,473,BTK,BTK,Bruton tyrosine kinase,BTK,TYR,Q06187,8ejb,NaN,14614,0,KELGTGQFGVVKYVAIKMIEFIEEAKVMMNLSEKLVQLYGVFIITE...,NaN,NaN,658,Q06187,MAAVILESIFLKRSQQKKKTSPLNFKKRLFLLTVHKLSYYEYDFER...
7,198,CDK2,CDK2,cyclin-dependent kinase 2,CDK2,CMGC,P24941,1oir,NaN,4389,0,EKIGEGTYGVVYKVALEKITAIREISLLKELNPNIVKLLDVYLVFE...,NaN,NaN,298,P24941,MENFQKVEKIGEGTYGVVYKARNKLTGEVVALKKIRLDTETEGVPS...
8,207,CDK9,CDK9,cyclin-dependent kinase 9,CDK9,CMGC,P50750,4or5,NaN,2233,0,AKIGQGTFGEVFKVALKKVTALREIKILQLLKENVVNLIEIYLVFD...,NaN,NaN,372,P50750,MAKQYDSVECPFCDEVSKYEKLAKIGQGTFGEVFKARHRKTGQKVA...
9,121,CHK1,CHK1,checkpoint kinase 1,CHEK1,CAMK,O14757,1zys,NaN,1655,0,QTLGEGAYGEVQLVAVKIVNIKKEICINAMLNENVVKFYGHYLFLE...,NaN,NaN,476,O14757,MAVPFVEDWDLVQTLGEGAYGEVQLAVNRVTEEAVAVKIVDMKRAV...


Calculate Kinase Features

In [12]:
df_kinases["structure.pocket"] = df_kinases["structure.pocket"].str.replace(r"[Xx_\s]", "", regex=True)
df_kinases.rename(columns={"kinase_name": "kinase"}, inplace=True)
pocket_seq = df_kinases[["kinase", "structure.pocket"]].dropna()
kinase_seq = df_kinases[["kinase", "Protein_Sequence"]].dropna()
#rename the index to kinase_id
kinase_seq.index.name = "kinase_id"
pocket_seq.index.name = "kinase_id"
pocket_seq


,kinase,structure.pocket
kinase_id,,
0,ABL,HKLGGGQYGEVYEVAVKTLEFLKEAAVMKEIKPNLVQLLGVYIITE...
1,AKT1,KLLGKGTFGKVILYAMKILHTLTENRVLQNSRPFLTALKYSCFVME...
2,ALK,RGLGHGAFGEVYEVAVKTLDFLMEALIISKFNQNIVRCIGVFILLE...
3,AurA,RPLGKGKFGNVYLLALKVLQLRREVEIQSHLRPNILRLYGYYLILE...
4,AXL,KTLGEGEFGAVMEVAVKTMDFLSEAVCMKEFDPNVMRLIGVVVILP...
5,BRAF,QRIGSGSFGTVYKVAVKMLAFKNEVGVLRKTRVNILLFMGYAIVTQ...
6,BTK,KELGTGQFGVVKYVAIKMIEFIEEAKVMMNLSEKLVQLYGVFIITE...
7,CDK2,EKIGEGTYGVVYKVALEKITAIREISLLKELNPNIVKLLDVYLVFE...
8,CDK9,AKIGQGTFGEVFKVALKKVTALREIKILQLLKENVVNLIEIYLVFD...


In [18]:
def get_aac(seq):
    return list(PyPro.GetProDes(seq).GetAAComp().values())

def get_dpc(seq):
    return list(PyPro.GetProDes(seq).GetDPComp().values())

def get_tpc(seq):
    return list(PyPro.GetProDes(seq).GetTPComp().values())

def get_mbauto(seq):
    return list(PyPro.GetProDes(seq).GetMoreauBrotoAuto().values())

def get_moranauto(seq):
    return list(PyPro.GetProDes(seq).GetMoranAuto().values())

def get_gearyauto(seq):
    return list(PyPro.GetProDes(seq).GetGearyAuto().values())

def get_ctd(seq):
    return list(PyPro.GetProDes(seq).GetCTD().values())

def get_socn(seq, maxlag=30):
    return list(PyPro.GetProDes(seq).GetSOCN(maxlag=maxlag).values())

def get_qso(seq, maxlag=30):
    return list(PyPro.GetProDes(seq).GetQSO(maxlag=maxlag).values())

def get_paac(seq, lamda=30):
    return list(PyPro.GetProDes(seq).GetPAAC(lamda=lamda).values())

def get_apaac(seq, lamda=30):
    return list(PyPro.GetProDes(seq).GetAPAAC(lamda=lamda).values())


In [19]:
kinase_seq["AAC"] = kinase_seq["Protein_Sequence"].apply(get_aac)
print("AAC done")
kinase_seq["DPC"] = kinase_seq["Protein_Sequence"].apply(get_dpc)
print("DPC done")
kinase_seq["TPC"] = kinase_seq["Protein_Sequence"].apply(get_tpc)
print("TPC done")
kinase_seq["MBauto"] = kinase_seq["Protein_Sequence"].apply(get_mbauto)
print("MBauto done")
kinase_seq["Moranauto"] = kinase_seq["Protein_Sequence"].apply(get_moranauto)
print("Moranauto done")
kinase_seq["Gearyauto"] = kinase_seq["Protein_Sequence"].apply(get_gearyauto)
print("Gearyauto done")
kinase_seq["CTD"] = kinase_seq["Protein_Sequence"].apply(get_ctd)
print("CTD done")
kinase_seq["SOCN"] = kinase_seq["Protein_Sequence"].apply(get_socn)
print("SOCN done")
kinase_seq["QSO"] = kinase_seq["Protein_Sequence"].apply(get_qso)
print("QSO done")
kinase_seq["PAAC"] = kinase_seq["Protein_Sequence"].apply(get_paac)
print("PAAC done")
kinase_seq["APAAC"] = kinase_seq["Protein_Sequence"].apply(get_apaac)
print("APAAC done")
print("Finished calculating features for full kinase sequences.")

pocket_seq["AAC"] = pocket_seq["structure.pocket"].apply(get_aac)
pocket_seq["DPC"] = pocket_seq["structure.pocket"].apply(get_dpc)
pocket_seq["TPC"] = pocket_seq["structure.pocket"].apply(get_tpc)
pocket_seq["MBauto"] = pocket_seq["structure.pocket"].apply(get_mbauto)
pocket_seq["Moranauto"] = pocket_seq["structure.pocket"].apply(get_moranauto)
pocket_seq["Gearyauto"] = pocket_seq["structure.pocket"].apply(get_gearyauto)
pocket_seq["CTD"] = pocket_seq["structure.pocket"].apply(get_ctd)
pocket_seq["SOCN"] = pocket_seq["structure.pocket"].apply(get_socn)
pocket_seq["QSO"] = pocket_seq["structure.pocket"].apply(get_qso)
pocket_seq["PAAC"] = pocket_seq["structure.pocket"].apply(get_paac)
pocket_seq["APAAC"] = pocket_seq["structure.pocket"].apply(get_apaac)
print("Finished calculating features for pocket sequences.")

#save 

AAC done
DPC done
TPC done
MBauto done
Moranauto done
Gearyauto done
CTD done
SOCN done
QSO done
PAAC done
APAAC done
Finished calculating features for full kinase sequences.
Finished calculating features for pocket sequences.


Calculate ASCII encoding

In [14]:
#create a dataframe with ascii values of pocket sequences
pocket_ascii = pocket_seq["structure.pocket"].apply(lambda x: [ord(a) for a in x])
pocket_ascii_df = pd.DataFrame(pocket_ascii.tolist(), index=pocket_seq.index)
pocket_ascii_df.columns = [str(i) for i in range(pocket_ascii_df.shape[1])]  # ensures strings
pocket_ascii_df.insert(0, "kinase", pocket_seq["kinase"])
pocket_ascii_df 


,kinase,0,1,2,3,4,5,6,7,8,...,75,76,77,78,79,80,81,82,83,84
kinase_id,,,,,,,,,,,,,,,,,,,,,
0,ABL,72,75,76,71,71,71,81,89,71,...,67,76,86,86,65,68,70,71.0,76.0,83.0
1,AKT1,75,76,76,71,75,71,84,70,71,...,76,77,76,73,84,68,70,71.0,76.0,67.0
2,ALK,82,71,76,71,72,71,65,70,71,...,67,76,76,73,71,68,70,71.0,77.0,65.0
3,AurA,82,80,76,71,75,71,75,70,71,...,76,76,76,73,65,68,70,71.0,87.0,83.0
4,AXL,75,84,76,71,69,71,69,70,71,...,67,77,76,86,65,68,70,71.0,76.0,83.0
5,BRAF,81,82,73,71,83,71,83,70,71,...,73,70,76,73,71,68,70,71.0,76.0,65.0
6,BTK,75,69,76,71,84,71,81,70,71,...,67,76,86,86,83,68,70,71.0,76.0,83.0
7,CDK2,69,75,73,71,69,71,84,89,71,...,76,73,76,65,68,70,71,76.0,65.0,NaN
8,CDK9,65,75,73,71,81,71,84,70,71,...,76,73,76,65,68,70,71,76.0,65.0,NaN


In [15]:
#replace updated data into existing excel file
from pandas import ExcelWriter

with ExcelWriter(dir_DATA+"ligand_and_kinase_features_data.xlsx", engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    mordred_feat_df.to_excel(writer, sheet_name="Ligand_Mordred_Features", index=False)
    morgan_fps_df.to_excel(writer, sheet_name="Ligand_Morgan_Fingerprints", index=False)
    
df_adj.to_pickle(dir_DATA+"Ligand_Adjacency_and_Feature_Matrices.pkl")

Save all the Features

In [21]:
#save all the created dataframes in seperate sheets in one excel file at dir_DATA

with pd.ExcelWriter(dir_DATA+"ligand_and_kinase_features_data.xlsx") as writer:
    kinase_seq.to_excel(writer, sheet_name="Kinase_Sequences_propy", index=False)
    pocket_seq.to_excel(writer, sheet_name="Kinase_Pocket_Sequences_propy", index=False)
    pocket_ascii_df.to_excel(writer, sheet_name="Kinase_ascii", index=False)
    mordred_feat_df.to_excel(writer, sheet_name="Ligand_Mordred_Features", index=False)
    morgan_fps_df.to_excel(writer, sheet_name="Ligand_Morgan_Fingerprints", index=False)   
    
#cannot save adjacency and feature matrices as excel, save as pickle
df_adj.to_pickle(dir_DATA+"Ligand_Adjacency_and_Feature_Matrices.pkl")

    